In [2]:
import ee
import folium
import geehydro
from IPython.display import Image

In [3]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [ ]:
def air_info_collection(location, year, month):
    """
    Obtain NO2 values for square around given location starting
    at current time and reaching back to the beginning of 2019.
    (At monthly intervals, pixel values are means for each month)

    :param country: Location of the polluter
    :param year: Current Year
    :param month: Current month

    :return float: sum of pixel values in the square, that are
                   above the threshold of XXX
    """

    square = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017"
    ).filter(ee.Filter.eq("country_na", str(country)))
    
    dataset = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2"
    ).select("tropospheric_NO2_column_number_density"
    ).filterDate(str(year)"-01-01", str(year+1)+"-01-01")
    
    def setProperty(image):
        dict = image.reduceRegion(ee.Reducer.sum(), square)
        return image.set(dict)

    pixel_values = dataset.map(setProperty)
    aggregate_pixel_values = (pixel_values.aggregate_array("tropospheric_NO2_column_number_density").getInfo())
    mean_pixel_value = np.mean(aggregate_pixel_values)

    return mean_pixel_value

In [ ]:
# iterating through months/years and locations of the given comapny
# to retrieve each NO2 value for the timeframe

month = 1
while i < len(working_countries):
    year = 2001
    while year <= 2020:
        result = ndvi_collection_usdos(country = working_countries[i], year = year)
        ndvi_values_usdos.append(result)
        country_index_usdos.append(working_countries[i])
        time_index_usdos.append(year)
        year += 1
    i += 1

In [40]:
lat = 51.266389
lng = 19.330556

coords = [lat, lng]
Map = folium.Map(location=coords, zoom_start=10)

square = ee.Geometry.BBox(lng-0.3, lat-0.2, lng+0.3, lat+0.2)

dataset = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2")
dataset = dataset.select('tropospheric_NO2_column_number_density')

date_filter = ee.Filter.date('2021-10-01', '2021-12-01')
dataset = dataset.filter(date_filter)

dataset = dataset.mean().clip(square)

visualisation_params = {
  "min": 0.00,
  "max": 0.0001,
  "palette": ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

Map.addLayer(square)
Map.addLayer(dataset, visualisation_params)

folium.Marker(
    location=[lat, lng],
    popup="Coal Power Plant",
    icon=folium.Icon(color="red", icon="info-sign"),
).add_to(Map)

Map